# Import Libraries

In [1]:
from swarmintelligence import GeneticAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/GA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_gens': 100, 'mr': 0.2, 'cr': 0.6}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['kapur_entropy']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    gwo_g = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    gwo_b = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
11.433972482012582  |  12.521042409172649  |  11.91254919127001
11.505368735994557  |  12.50376318208045  |  12.09464112073585
11.329344627410823  |  12.519888830487073  |  12.111721378324733
11.400614079260102  |  12.501649137716267  |  12.134302331142313
11.492311443507816  |  12.525963749213673  |  12.070244371750341
11.486949662630604  |  12.449964941468817  |  12.018303296839646
11.50934324535306  |  12.301795042374394  |  12.078068649147825
11.392889290149036  |  12.506880298004535  |  12.069331546581026
11.403548532788697  |  12.512032254810636  |  12.026205301913476
11.273829853472057  |  12.457480515054431  |  12.10215397918851
11.31962483767863  |  12.398372593129704  |  12.089319270694247
11.300421541619574  |  12.40714083963744  |  12.055127254742342
11.210615074553779  |  12.443995672065205  |  12.008388635171139
11.457074342653092  |  12.503361870383163  |  12.118122603730068
11.397352535540065  |  12.445760740041766  |  12.123911542951

Mean Fitness = 20.050119085466395
	 Lena.png | Threshold =  2
11.608539568073235  |  12.67484231581684  |  12.044026856882237
11.609335378138388  |  12.415060807164673  |  11.986512202228475
11.556180429319914  |  12.55928298377626  |  11.968225105509259
11.599646555691452  |  12.51834911032498  |  12.062729785651143
11.606940971838634  |  12.653488126661571  |  11.860581888787019
11.581195126158057  |  12.588501792418183  |  12.061414879244738
11.474890915468366  |  12.520887121726975  |  12.035717882749772
11.517739540204069  |  12.66730523020168  |  12.034501328374398
11.565177524627646  |  12.686991235957915  |  11.986873303308954
11.570785273880025  |  12.685807167212726  |  12.049075084872104
11.560099484398085  |  12.571904609632096  |  11.988092838680853
11.550590022453305  |  12.647848067056197  |  11.97669617312486
11.575188803313713  |  12.641487523088333  |  12.067730452290986
11.451993660456324  |  12.689521271676252  |  12.04713164459538
11.595454302377863  |  12.66506086

Mean Fitness = 19.650138755157435
	 Male.tiff | Threshold =  2
12.585395415503946  |  12.578505924704684  |  12.200090606026492
12.628954274375896  |  12.505731818185101  |  12.477205523613026
12.626384291393899  |  12.151094400530223  |  12.426592856057738
12.413304992433522  |  12.566420678272888  |  12.595071976301512
12.502697689144158  |  12.63137629666482  |  12.627494532358996
12.45842840917065  |  12.46185487810152  |  12.35846084516021
12.571602127163395  |  12.612172998830985  |  12.489905996596242
12.611839484719239  |  12.626384291393899  |  12.483614187294403
12.612245871995581  |  12.623066758307283  |  12.08936502034551
11.936250350496888  |  12.330480801373751  |  12.418875562138632
12.628954274375896  |  12.47167171770697  |  12.312677795280964
12.632518082620024  |  12.391939381085948  |  12.627345375084591
12.626017232471877  |  12.462917781776863  |  12.45275214798745
12.427470142030177  |  12.434875966656135  |  12.634792970346426
12.251697101914521  |  12.59540423

Mean Fitness = 20.905307827829755
	 Mandrill.tiff | Threshold =  2
12.841977869936587  |  12.357034726257062  |  12.773615911388
12.927651526888795  |  12.329640483194918  |  12.79369337168358
12.880181227023844  |  12.269586962362794  |  12.762955951530415
12.873900849772703  |  12.26717196953896  |  12.790912807076415
12.882127023005868  |  12.225434441689806  |  12.756251355870644
12.913675531789512  |  12.247032413880373  |  12.774499250274921
12.88002474899929  |  12.25023658759312  |  12.740151432912926
12.834716732898418  |  12.2395250239326  |  12.662450845511708
12.927629591266358  |  12.346353416630409  |  12.689822219224158
12.946092878485516  |  12.244180670463038  |  12.772373950044802
12.932305615632691  |  12.374212636610434  |  12.756300223230443
12.83075114401069  |  12.308111700677781  |  12.636637074235502
12.919599473125523  |  12.265018324802826  |  12.782149292344123
12.801942663492586  |  12.241601873808948  |  12.771237662999447
12.935634325501217  |  12.3562333

Mean Fitness = 21.077178280785027
	 Peppers.tiff | Threshold =  2
12.396883263139522  |  12.552500799002607  |  12.372695917659673
12.343466029881982  |  12.624514342284618  |  12.219509943082766
12.429234893561908  |  12.579300081630139  |  12.497649849301977
12.431522584801623  |  12.57637624762421  |  12.509942537689334
12.374758234475816  |  12.625349099827861  |  12.469478228848903
12.32348018929717  |  12.561942756235924  |  12.33626952605589
12.446837159033738  |  12.500142479523308  |  12.475757229426343
12.459502059526969  |  12.534685186242553  |  12.457173279815514
12.469673811678604  |  12.422205464727163  |  12.493993655931177
12.372311970811847  |  12.391477378434473  |  12.290880216187784
12.409267234279525  |  12.615771575384548  |  12.337279499591483
12.236976275375916  |  12.54930275111549  |  12.321285715148033
12.463596738884863  |  12.551492132327011  |  12.104363924063202
12.43612041843449  |  12.575361252250485  |  12.405889586169703
12.451896789202396  |  12.500

Mean Fitness = 20.577696157567004
	 Sailboat on lake.tiff | Threshold =  2
12.326991886947283  |  12.83980449756073  |  11.957063270397189
12.31524221682558  |  12.849128735417786  |  12.05764168423118
12.28253242896767  |  12.893382637318734  |  12.081232488454333
12.284722673057777  |  12.712301672806788  |  11.961108600173231
12.27974182081069  |  12.885277796364107  |  11.896570536307141
12.233000265219037  |  12.905129870030207  |  12.046433786600506
12.214339803845265  |  12.779100409772138  |  12.0097521318986
12.253133043917448  |  12.822746235864098  |  12.00200358348136
12.311290012033057  |  12.891730215353729  |  12.00394343864185
12.35201978362625  |  12.90832964105487  |  12.011903237875494
12.241126368931443  |  12.899410139195467  |  11.88144966172424
12.347762460736234  |  12.848240824101687  |  11.744532669166153
12.110810760804547  |  12.87983546882921  |  11.913051444533323
12.262716201351745  |  12.904941468436977  |  12.023029026095434
12.095183448668134  |  12.72

Mean Fitness = 20.67122138761466


## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,kapur_entropy,max,"[88, 168]","[82, 173]","[113, 172]",11.386362,12.448410,12.059400,...,"[[113, 255, 255, 255, 255, 255, 255, 255, 255,...","[11.433972482012582, 11.505368735994557, 11.32...","[12.521042409172649, 12.50376318208045, 12.519...","[11.91254919127001, 12.09464112073585, 12.1117...","[7.487243175506592, 8.6593918800354, 6.9938933...","[2858.3773028055825, 2665.611155192057, 2765.9...","[53.46379431732827, 51.629556992018216, 52.592...","[13.569608062130687, 13.872835636786501, 13.71...","[0.7198968193842304, 0.7505642623537264, 0.747...","[0.9256356970875724, 0.9298686071633986, 0.928..."
1,AirplaneF16.tiff,3,kapur_entropy,max,"[73, 124, 175]","[62, 121, 175]","[75, 130, 178]",14.346652,15.496620,15.096085,...,"[[130, 255, 255, 255, 255, 255, 255, 255, 255,...","[14.553311561955605, 14.377700158715767, 14.25...","[15.515429628060629, 15.221708811682392, 15.64...","[15.229002909231166, 15.015309278951973, 15.29...","[10.493349313735962, 7.849551439285278, 10.091...","[2235.368512471517, 2461.5485623677573, 2297.4...","[47.27968392947987, 49.613995629940526, 47.931...","[14.637312317264875, 14.21871952747637, 14.518...","[0.7905963037683429, 0.7677013756232082, 0.777...","[0.9452618821500058, 0.9414964406860274, 0.944..."
2,AirplaneF16.tiff,4,kapur_entropy,max,"[67, 106, 145, 181]","[60, 99, 146, 183]","[67, 112, 143, 181]",16.969933,18.275072,17.596416,...,"[[112, 255, 255, 255, 255, 255, 255, 255, 255,...","[16.368236360760868, 17.032835180310535, 16.88...","[18.202177700059394, 18.000899046229623, 18.52...","[17.76697127591037, 17.80356523098586, 17.2854...","[8.170486211776733, 10.291805028915405, 7.6656...","[1663.4701703389485, 2031.4229011535645, 1885....","[40.78566133261723, 45.07130906855895, 43.4177...","[15.920653434438599, 15.052800167252752, 15.37...","[0.7381111517008176, 0.7981565032050782, 0.768...","[0.9577396564139318, 0.9518069712342904, 0.953..."
3,AirplaneF16.tiff,5,kapur_entropy,max,"[57, 86, 124, 152, 183]","[43, 76, 118, 155, 188]","[59, 98, 133, 164, 191]",19.254399,20.884190,20.011767,...,"[[133, 255, 255, 255, 255, 255, 255, 255, 255,...","[19.223365278172697, 19.039398946249833, 19.22...","[21.056702141895236, 20.983553793698583, 21.16...","[19.41641002913847, 19.986770597773443, 20.052...","[9.836575984954834, 9.40766954421997, 7.367010...","[2053.594324747721, 1388.081621805827, 1963.59...","[45.31660098405132, 37.25696742631943, 44.3124...","[15.005657054061954, 16.70665356667193, 15.200...","[0.7987436821103079, 0.7408768263089551, 0.793...","[0.9535474321038677, 0.967383413381666, 0.9540..."
4,Lena.png,2,kapur_entropy,max,"[136, 187]","[75, 148]","[106, 152]",11.542600,12.607673,11.992886,...,"[[152, 152, 152, 152, 152, 152, 152, 152, 152,...","[11.608539568073235, 11.609335378138388, 11.55...","[12.67484231581684, 12.415060807164673, 12.559...","[12.044026856882237, 11.986512202228475, 11.96...","[8.082986831665039, 7.414580821990967, 7.42780...","[2317.499656677246, 2573.2717628479004, 2007.8...","[48.14041604179638, 50.72742614057903, 44.8091...","[14.480606823893211, 14.025947064598556, 15.10...","[0.6340169007355135, 0.6211780795495669, 0.606...","[0.8903944753864984, 0.8859536782165623, 0.882..."
5,Lena.png,3,kapur_entropy,max,"[111, 157, 203]","[68, 121, 173]","[94, 130, 174]",14.178448,15.503717,14.768243,...,"[[130, 130, 174, 130, 130, 130, 130, 130, 130,...","[13.6510201479085, 13.78744559248287, 14.35070...","[15.488523421966292, 15.251708086011718, 15.58...","[14.823956122476634, 14.544313409003177, 14.90...","[9.303286790847778, 9.251329898834229, 9.03344...","[1401.643502553304, 1636.5992075602214, 1457.5...","[37.438529652662695, 40.45490338092802, 38.178...","[16.664427927162382, 15.99138024312756, 16.494..

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/GA/GA_kapur_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/GA/GA_kapur_RGB_dict.pkl', 'wb'))